# Advent of Code 2021

## Day 15

Task:

* Find the shortest route given an array of travel costs

Reflection:

This is the first AOC solution I'm really proud of. The clever bit is modeling the grid of travel cost values as a graph where the costs are the node, then creating a line graph from that graph so that the costs are represented by edges. Then a single function can find the weighted length of the shortest path from start to end.

In [87]:
import aoc
import numpy as np
import networkx as nx
aoc.read_input("cookie.txt", "input_2021", 2021, 15)    

### puzzle 1

In [88]:
with open("input_2021/15.txt") as f:
    data = [[int(char) for char in line] for line in f.read().splitlines()]
    dataArray = np.array(data)

# Input grid represents edges
# Create grid graph of equal size, then convert nodes to edges (and edges to nodes)
xmax = dataArray.shape[1] - 1
ymax = dataArray.shape[0] - 1
G = nx.grid_graph(dim=(xmax+1, ymax+1))
Gprime = nx.line_graph(G)

# calculate edge weights from input array
for e in Gprime.edges():
    j, i = set(e[0]).intersection(set(e[1])).pop()
    Gprime[e[0]][e[1]]["weight"] = dataArray[i, j]

# add start and stop nodes
# start node has 0 weight b/c it is free
Gprime.add_edge("start", ((0, 0), (1, 0)), weight=0)
Gprime.add_edge("start", ((0, 0), (0, 1)), weight=0)
Gprime.add_edge("end", ((xmax-1, ymax), (xmax, ymax)), weight=dataArray[ymax, xmax])
Gprime.add_edge("end", ((xmax, ymax-1), (xmax, ymax)), weight=dataArray[ymax, xmax])

# calculate length of the shortest path
nx.shortest_path_length(Gprime, "start", "end", "weight")

410

### puzzle 2

In [112]:
with open("input_2021/15.txt") as f:
    data = [[int(char) for char in line] for line in f.read().splitlines()]
    dataArray = np.array(data)

# multiply input data
bigDataArray = dataArray.copy()
for i in range(4):
    dataArray = (dataArray.copy() % 9) + 1
    bigDataArray = np.concatenate((bigDataArray, dataArray))

dataArray = bigDataArray.copy()
for i in range(4):
    dataArray = (dataArray.copy() % 9) + 1
    bigDataArray = np.concatenate((bigDataArray, dataArray), axis=1)

# Input grid represents edges
# Create grid graph of equal size, then convert nodes to edges (and edges to nodes)
xmax = bigDataArray.shape[1] - 1
ymax = bigDataArray.shape[0] - 1
G = nx.grid_graph(dim=(xmax+1, ymax+1))
Gprime = nx.line_graph(G)

# calculate edge weights from input array
for e in Gprime.edges():
    j, i = set(e[0]).intersection(set(e[1])).pop()
    Gprime[e[0]][e[1]]["weight"] = bigDataArray[i, j]


# add start and stop nodes
# start node has 0 weight b/c it is free
Gprime.add_edge("start", ((0, 0), (1, 0)), weight=0)
Gprime.add_edge("start", ((0, 0), (0, 1)), weight=0)
Gprime.add_edge("end", ((xmax-1, ymax), (xmax, ymax)), weight=bigDataArray[ymax, xmax])
Gprime.add_edge("end", ((xmax, ymax-1), (xmax, ymax)), weight=bigDataArray[ymax, xmax])

# calculate length of the shortest path
nx.shortest_path_length(Gprime, "start", "end", "weight")

2809

## Day 14

In [894]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 14)

### puzzle 1

In [967]:
with open("input_2021/14.txt") as f:
    lines = f.read().splitlines()
    template = lines.pop(0)
    rules = {rule[:2]: rule[-1] + rule[1] for rule in lines[1:]}


def polymerize(template, rules, step, current_step=0):
    new_template = template[:1]
    current_step += 1
    for i, char in enumerate(template[:-1]):
        try: # sequence is found in rules
            new_seq = rules[template[i:i+2]]
            new_template += new_seq
        except: # sequence not found
            continue
    if current_step < step:
        polymer = polymerize(new_template, rules, step, current_step)
    else: polymer = new_template
    return polymer

polymer = polymerize(template, rules, 10)
elements = {}
for char in polymer:
    try:
        elements[char] += 1
    except:
        elements[char] = 1

max(elements.values()) - min(elements.values())

2233

## puzzle 2

Above approach is fine for 20 steps or so. But the time complexity is not good because the template is nearly doubling in length every time.

It isn't necessary to reproduce the polymer, only to count how many of each letter there is. Count how many matching pairs there are, then increase each matching pair each iteration.

consider template ababab and rules ab -> c, ac -> d, and cb -> e. In the template polymer, the counts of a, b and ab are all 3, and the counts of ac, cb, c, and d are 0. For each occurance of ab, decrease the count of ab by 1, and increase the count of ac, cb, and c by 1.

after step 1, resulting polymer string is acbacbacb. The count of ab is 0 (3 - 3), the counts of ac, cb, and c are 3 (0 + 3), and the counts of a and b are also 3 (3 + 0)

after step 2, the resulting polymer string is adcebadcebadceb. The counts of ac and cb are 0 (3 - 3), the counts of ad, dc, ce, and eb are 3 (0 + 3). The counts of d and e increase by 3, the counts of a, b, and c do not not change.

In [1042]:
with open("input_2021/14.txt") as f:
    lines = f.read().splitlines()

    # model template a dictionary with count of all 1 and 2 letter combinations
    templateString = lines.pop(0)
    rules = {rule[:2]: rule[-1] for rule in lines[1:]}

# model template as two dictionarys with counts of all 1 and 2 letter combinations
templateDict1 = {}
for char in templateString:
    try:
        templateDict1[char] += 1
    except:
        templateDict1[char] = 1
templateDict2 = {k: 0 for k in rules}
for i, char in enumerate(templateString[:-1]):
    seq = templateString[i:i+2]
    templateDict2[seq] += 1

def polymerize(templateDict1, templateDict2, rules, step, current_step=0):
    new_templateDict1 = templateDict1.copy()
    new_templateDict2 = templateDict2.copy()
    
    current_step += 1
    for k, v in templateDict2.items():
        if templateDict2[k] > 0:
            char = rules[k]
            try:
                new_templateDict1[char] += v
            except:
                new_templateDict1[char] = v
            seq1 = k[0]+char
            seq2 = char+k[1]
            new_templateDict2[seq1] += v
            new_templateDict2[seq2] += v
            new_templateDict2[k] -= v
        
    if current_step < step:
        polymer = polymerize(new_templateDict1, new_templateDict2, rules, step, current_step)
    else: polymer = new_templateDict1
    return polymer

polymerCharCount = polymerize(templateDict1, templateDict2, rules, 40)
max(polymerCharCount.values()) - min(polymerCharCount.values())

2884513602164

## Day 13

In [890]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 13)

# Parse data
with open("input_2021/13.txt") as f:
    dots, instructions = f.read().strip().split("\n\n")
    dots = [[int(dot) for dot in dotCoords.split(",")] for dotCoords in dots.split("\n")]
    instructions = instructions.split("\n")
    instructionsList = []
    for i in instructions:
        a, n = i.split('=')
        a = a[-1]
        n = int(n)
        instructionsList.append([a, n])

### puzzle 1

In [892]:
# assuming every fold is eaxctly in half to predict the size of the initial array
# hard coding first fold is on x axis
# swap the first subscript values on the instructionsList variable if first fold is on y axis

import numpy as np
dataArray = np.zeros((instructionsList[1][1]*2 + 1, instructionsList[0][1]*2 + 1))

for dot in dots:
    dataArray[dot[1], dot[0]] = 1

for instruction in instructionsList[:1]:
    axis = 0 if instruction[0] == 'y' else 1
    trimmed_array = np.delete(dataArray, instruction[1], axis)
    arr1, arr2 = np.split(trimmed_array, 2, axis)
    arr2 = np.flip(arr2, axis)
    dataArray = arr1 + arr2
(dataArray > 0).sum()

781

### puzzle 2

In [893]:
dataArray = np.zeros((instructionsList[1][1]*2 + 1, instructionsList[0][1]*2 + 1))
for dot in dots:
    dataArray[dot[1], dot[0]] = 1

for instruction in instructionsList:
    axis = 0 if instruction[0] == 'y' else 1
    trimmed_array = np.delete(dataArray, instruction[1], axis)
    arr1, arr2 = np.split(trimmed_array, 2, axis)
    arr2 = np.flip(arr2, axis)
    dataArray = arr1 + arr2

# transform into readable output
dataArrayInt = (dataArray > 0).astype(int)
for row in dataArrayInt:
    rowChars = ["██" if cell else "  " for cell in row]
    print("".join(rowChars))

██████    ████████  ██████      ████      ████        ████  ██████    ██████    
██    ██  ██        ██    ██  ██    ██  ██    ██        ██  ██    ██  ██    ██  
██    ██  ██████    ██    ██  ██        ██              ██  ██    ██  ██████    
██████    ██        ██████    ██        ██  ████        ██  ██████    ██    ██  
██        ██        ██  ██    ██    ██  ██    ██  ██    ██  ██        ██    ██  
██        ████████  ██    ██    ████      ██████    ████    ██        ██████    


## Day 12

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 12)

### puzzle 1

In [ ]:
# this is a graph problem. Use networkx
import networkx as nx
edges = []
with open("input_2021/12_test.txt") as f:
    for line in f:
        edges.append(line.strip().split("-"))

G = nx.Graph()
G.add_edges_from(edges)

In [ ]:
visitedList = [[]]

def depthFirst(graph, currentVertex, visited):
    visited.append(currentVertex)
    for vertex in graph.neighbors(currentVertex):
        if vertex not in visited or vertex.isupper():
            depthFirst(graph, vertex, visited.copy())
    visitedList.append(visited)

depthFirst(G, "start", [])

len([path for path in visitedList if len(path) > 0 and path[-1] == 'end'])


### puzzle 2

Originally, a version of this code that I'm pretty sure would have worked ran for over 13 mins without completing. Partly that may have been because I was converting a list into a set on every recurse of the function, and partly I think it was due to copying the networkx graph at every recurse. 

I converted the networkx graph to a dictionary and move the set operation out of the function. The code completed in less than 1 minute.

In [ ]:
# this is a graph problem. Use networkx
import networkx as nx
edges = []
with open("input_2021/12.txt") as f:
    for line in f:
        edges.append(line.strip().split("-"))

G = nx.Graph()
G.add_edges_from(edges)
graph = {n:{"neighbors": [nb for nb in G.neighbors(n)]} for n in G.nodes}

In [ ]:
import copy
def depthFirst(G, currentVertex, visited, visitedList):
    visited.append(currentVertex)
    G[currentVertex]["remaining"] -= 1
    visitable = [n for n in G[currentVertex]["neighbors"] if G[n]["remaining"] > 0]
    for vertex in  visitable:
        if vertex == 'end':
            visited.append(vertex)
        else:
            depthFirst(copy.deepcopy(G), vertex, visited.copy(), visitedList)
    visitedList.append(visited)
    
    return visitedList

lowers = [n for n in G.nodes if n.islower() and n not in ("start", "end")]
all_paths = set()

for chosen in lowers:
    for k, v in graph.items():
        if k.isupper():
            v["remaining"] = 100000
        elif k == chosen:
            v["remaining"] = 2
        else:
            v["remaining"] = 1
    visitedList = depthFirst(graph, "start", [], [])
    visitedSet = set([",".join(path) for path in visitedList if path[-1] == 'end'])
    #print(chosen, "done")
    all_paths = all_paths | visitedSet

#print(graph)
print(len(all_paths))
#all_paths

## Day 11

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 11)

### puzzle 1

In [ ]:
import numpy as np

# read data into array & buffer by 1 cell of nan values
# to avoid edge problems when checking neighbors
with open("input_2021/11_test.txt") as f:
    data = [[float(char) for char in seq] for seq in f.read().strip().split("\n")]
    data = np.pad(data, pad_width=1, mode='constant', constant_values=np.nan)
dataArray = np.array(data)
print(dataArray)

# track flashes
flashCount = 0
for i in range(100):
    dataArray += 1
    flashingOctopusCoords = [(y, x) for y, x in zip(*np.where(dataArray > 9))]
    
    
    while flashingOctopusCoords:
        for coord in flashingOctopusCoords:
            flashCount += 1
            
            dataArray[coord] = 0
            # add 1 to all neighbors
            dataArray[(coord[0]-1, coord[1]-1)] += int(dataArray[(coord[0]-1, coord[1]-1)] > 0)
            dataArray[(coord[0]-1, coord[1])] += int(dataArray[(coord[0]-1, coord[1])]  > 0)
            dataArray[(coord[0]-1, coord[1]+1)] += int(dataArray[(coord[0]-1, coord[1]+1)] > 0)
            dataArray[(coord[0], coord[1]+1)] += int(dataArray[(coord[0], coord[1]+1)] > 0)
            dataArray[(coord[0]+1, coord[1]+1)] += int(dataArray[(coord[0]+1, coord[1]+1)] > 0)
            dataArray[(coord[0]+1, coord[1])] += int(dataArray[(coord[0]+1, coord[1])] > 0)
            dataArray[(coord[0]+1, coord[1]-1)] += int(dataArray[(coord[0]+1, coord[1]-1)] > 0)
            dataArray[(coord[0], coord[1]-1)] += int(dataArray[(coord[0], coord[1]-1)] > 0)
            
            

        flashingOctopusCoords = [(y, x) for y, x in zip(*np.where(dataArray > 9))]

    
flashCount



check each for flash, add to neighbors if flash
repeat until no flash (while loop, something like checkFlash = True)

### puzzle 2

In [ ]:
import numpy as np

# read data into array & buffer by 1 cell of nan values
# to avoid edge problems when checking neighbors
with open("input_2021/11.txt") as f:
    data = [[float(char) for char in seq] for seq in f.read().strip().split("\n")]
    data = np.pad(data, pad_width=1, mode='constant', constant_values=np.nan)
dataArray = np.array(data)
print(dataArray)

# track flashes
flashCount = 0
not_all_flash = True
counter = 0
while not_all_flash:
    counter += 1
    dataArray += 1
    flashingOctopusCoords = [(y, x) for y, x in zip(*np.where(dataArray > 9))]
    
    
    while flashingOctopusCoords:
        for coord in flashingOctopusCoords:
            flashCount += 1
            
            dataArray[coord] = 0
            # add 1 to all neighbors
            dataArray[(coord[0]-1, coord[1]-1)] += int(dataArray[(coord[0]-1, coord[1]-1)] > 0)
            dataArray[(coord[0]-1, coord[1])] += int(dataArray[(coord[0]-1, coord[1])]  > 0)
            dataArray[(coord[0]-1, coord[1]+1)] += int(dataArray[(coord[0]-1, coord[1]+1)] > 0)
            dataArray[(coord[0], coord[1]+1)] += int(dataArray[(coord[0], coord[1]+1)] > 0)
            dataArray[(coord[0]+1, coord[1]+1)] += int(dataArray[(coord[0]+1, coord[1]+1)] > 0)
            dataArray[(coord[0]+1, coord[1])] += int(dataArray[(coord[0]+1, coord[1])] > 0)
            dataArray[(coord[0]+1, coord[1]-1)] += int(dataArray[(coord[0]+1, coord[1]-1)] > 0)
            dataArray[(coord[0], coord[1]-1)] += int(dataArray[(coord[0], coord[1]-1)] > 0)
            
            

        flashingOctopusCoords = [(y, x) for y, x in zip(*np.where(dataArray > 9))]
    
    if np.all(dataArray[1:10, 1:10] == 0):
        not_all_flash = False
        print(counter)

In [ ]:
np.all(dataArray[1:10, 1:10] == 5)

## Day 10

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 10)

### puzzle 1

In [ ]:
score = 0
with open("input_2021/10.txt") as f:
    for line in f:
        seq = line.strip()
        openChars = ["(", "[", "{", "<"]
        legalClose = []
        for char in seq:
            if char in openChars or char == legalClose[-1]:
                if char == "(":
                    legalClose.append(")")
                elif char == "[":
                    legalClose.append("]")
                elif char == "{":
                    legalClose.append("}")
                elif char == "<":
                    legalClose.append(">")
                else:
                    legalClose.pop(-1)
                continue
            else:
                if char == ")":
                    score += 3
                if char == "]":
                    score += 57
                if char == "}":
                    score += 1197
                if char == ">":
                    score += 25137
                break
score




### puzzle 1 refactor

In [ ]:
score = 0
with open("input_2021/10.txt") as f:
    for line in f:
        seq = line.strip()
        charDict = {
            "(": [")", 3], 
            "[": ["]", 57], 
            "{": ["}", 1197], 
            "<": [">", 25137]
            }
        scoreDict = {item[1][0]: item[1][1] for item in charDict.items()}
        legalClose = []
        for char in seq:
            if char in charDict.keys():
                legalClose.append(charDict[char][0])
                 
            elif char == legalClose[-1]:
                legalClose.pop(-1)
                continue
            else:
                score += scoreDict[char]
                break
score

### puzzle 2

In [ ]:
scores = []
with open("input_2021/10.txt") as f:
    for line in f:
        seq = line.strip()
        charDict = {
            "(": [")", 1], 
            "[": ["]", 2], 
            "{": ["}", 3], 
            "<": [">", 4]
            }
        scoreDict = {item[1][0]: item[1][1] for item in charDict.items()}
        legalClose = []
        corrupt = False
        for char in seq:
            if char in charDict.keys():
                legalClose.append(charDict[char][0])
                 
            elif char == legalClose[-1]:
                legalClose.pop(-1)
                continue
            else:
                corrupt = True
                break
        if not corrupt:
            closing = legalClose[::-1]
            score = 0
            for char in closing:
                score *= 5
                score += scoreDict[char]
            scores.append(score)
scores.sort()
scores[len(scores)//2]


## Day 9

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 9)

### puzzle 1

In [ ]:
import numpy as np
total_risk = 0
with open("input_2021/9.txt") as f:
    input = [[int(digit) for digit in num] for num in f.read().strip().split("\n")]
    heightMap = np.array(input)
    heightMap = np.pad(heightMap, pad_width=1, mode='constant', constant_values=100)

height, width = heightMap.shape
for i in range(1, height-1):
    for j in range(1, width-1):
        val = heightMap[i, j]
        neighbors = [heightMap[i+1, j], heightMap[i-1, j], heightMap[i, j+1], heightMap[i, j-1]]
        if val < min(neighbors):
            risk = val + 1
            total_risk += risk
total_risk


### puzzle 2

In [ ]:
import scipy

# Read input into numpy array
with open("input_2021/9.txt") as f:
    input = [[int(digit) for digit in num] for num in f.read().strip().split("\n")]
    heightMap = np.array(input)

# The scipy label function will identify regions.
# Anything with a non-zero value is part of a region
# So change the underlying array so that all 9's are read as zeros
height, width = heightMap.shape
for i in range(height):
    for j in range(width):
        if heightMap[i, j] < 9:
            heightMap[i, j] = 1
        else: 
            heightMap[i, j] = 0

labeled_array, num_features = scipy.ndimage.label(heightMap)
labeled_array, num_features

# Count the size of each unique region
unique, counts = np.unique(labeled_array, return_counts=True)

# Remove the count for the 0-region (i.e. the border between regions)
basin_sizes = [val[1] for val in zip(unique, counts) if val[0] != 0]

# Get the top three and multiply them together
a, b, c  = sorted(basin_sizes)[-3:]
a * b * c


## Day 8

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 8)

### puzzle 1

In [ ]:
num_count = 0
with open("input_2021/8.txt") as f:
    for line in f:
        output = line.strip().split(" | ")[-1].split()
        for val in output:
            if len(val) in [2, 3, 4, 7]:
                num_count += 1
num_count

### puzzle 2

In [ ]:
total_val = 0
with open("input_2021/8.txt") as f:
    for line in f:
        numDict = {}
        inputs, outputs = [vals.split() for vals in line.strip().split(" | ")]
        for num in inputs:
            if len(num) == 2:
                numDict[1] = num
            elif len(num) == 3:
                numDict[7] = num
            elif len(num) == 4:
                numDict[4] = num
            elif len(num) == 7:
                numDict[8] = num
        
        inputs = [e for e in inputs if e not in numDict.values()]
        fives = [e for e in inputs if len(e) == 5]
        sixes = [e for e in inputs if len(e) == 6]

        # 6 is the only 6-line number that intersects only one line in 1
        for num in sixes:
            if len(set.intersection(set(num), set(numDict[1]))) == 1:
                numDict[6] = num
        sixes.remove(numDict[6])

        # 3 is the only 5-line number that intersects both lines in 1
        for num in fives:
            if len(set.intersection(set(num), set(numDict[1]))) == 2:
                numDict[3] = num
        fives.remove(numDict[3])

        for num in sixes:
            if len(set.intersection(set(num), set(numDict[3]))) == 4:
                numDict[0] = num
            else: 
                numDict[9] = num

        for num in fives:
            if len(set.intersection(set(num), set(numDict[4]))) == 2:
                numDict[2] = num
            else:
                numDict[5] = num

        codeDict = {v:k for k, v in numDict.items()}

        decrypted = ''
        for num in outputs:
            for k in codeDict.keys():
                if set(num) == set(k):
                    decrypted += str(codeDict[k])
        total_val += int(decrypted)

total_val
    

In [ ]:
numDict.values()

In [ ]:
testLine = "acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf"
inputs, outputs = [vals.split() for vals in testLine.strip().split(" | ")]
numDict = {}
# get known
for num in inputs:
    if len(num) == 2:
        numDict[1] = num
    elif len(num) == 3:
        numDict[7] = num
    elif len(num) == 4:
        numDict[4] = num
    elif len(num) == 7:
        numDict[8] = num
inputs = [e for e in inputs if e not in numDict.values()]
fives = [e for e in inputs if len(e) == 5]
sixes = [e for e in inputs if len(e) == 6]

# 6 is the only 6-line number that intersects only one line in 1
for num in sixes:
    if len(set.intersection(set(num), set(numDict[1]))) == 1:
        numDict[6] = num
sixes.remove(numDict[6])

# 3 is the only 5-line number that intersects both lines in 1
for num in fives:
    if len(set.intersection(set(num), set(numDict[1]))) == 2:
        numDict[3] = num
fives.remove(numDict[3])

for num in sixes:
    if len(set.intersection(set(num), set(numDict[3]))) == 4:
        numDict[0] = num
    else: 
        numDict[9] = num

for num in fives:
    if len(set.intersection(set(num), set(numDict[4]))) == 2:
        numDict[2] = num
    else:
        numDict[5] = num


print(numDict)
codeDict = {v:k for k, v in numDict.items()}
print(codeDict)
decrypted = ''
for num in outputs:
    for k in codeDict.keys():
        if set(num) == set(k):
            decrypted += str(codeDict[k])
decrypted

segment is in 1, but not 7, that's top
5 segment numbers: 2, 3, 5
6 segment numbers: 6, 9, 0


5 & 6 don't have upper right
1 & 4 don't have top
1, 7, and 0 don't have mid
1, 2, 3, and 7 don't have upper left
1, 3, 4, 5, 7, 9 don't have lower left
1, 4, 7 don't have bottom
2 doesn't have lower right
segment is in 9, but not 4 + top, that's bottom
segment is in 3, but not 1 + top + bottom, that's middle
segment is in 8, but not 0, that's the middle
segment is in 8, but not 6, that's upper right
segment is in 8, but not 9, that's lower left


## Day 7

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 7)

### puzzle 1

In [ ]:
import statistics
with open("input_2021/7.txt") as f:
    positions = [int(val) for val in f.read().strip().split(",")]
    center = statistics.median(positions)
    fuel_cost = sum([abs(position - center) for position in positions])
fuel_cost

### puzzle 2

Interesting that the absolutely correct answer is using the mean for the center. But If that isn't an integer value, you can't just round. In this case, the rounded value was 485. But that gives a slightly worse response than 484. Even though using the mean gave a better response than 484.

In [ ]:
def fuel(n):
    return n*(n+1)/2

with open("input_2021/7.txt") as f:
    positions = [int(val) for val in f.read().strip().split(",")]
    center = statistics.mean(positions)
    fuel_cost = sum([fuel(abs(position - center)) for position in positions])
fuel_cost

In [ ]:
def fuel(n):
    return n*(n+1)/2

with open("input_2021/7.txt") as f:
    positions = [int(val) for val in f.read().strip().split(",")]
    min_pos = min(positions)
    max_pos = max(positions)
    fuel_costs = []
    for center in range(min_pos, max_pos+1):
        fuel_cost = sum([fuel(abs(position - center)) for position in positions])
        fuel_costs.append(fuel_cost)
min(fuel_costs)

## Day 6

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 6)

### puzzle 1

In [ ]:
def fish_tracker(fishList, day):
    newFishList = fishList[:]
    for i, fish in enumerate(fishList):
        if fish - 1 >= 0:
            newFishList[i] -= 1
        else:
            newFishList[i] = 6
            newFishList.append(8)
    if day == 80:
        return len(newFishList)
    else:
        result = fish_tracker(newFishList, day + 1)
    return result
        


with open("input_2021/6.txt") as f:
    day = 1
    fishList = [int(fish) for fish in f.read().strip().split(',')]
    count = fish_tracker(fishList, day)
print(count)
    

### puzzle 2

Just changing the days to 256 doesn't work because of the exponential growth. track individual cohorts instead. After the initial period, move new fish to an existing cohort (for example, on day 7, move the new cohort 8 fish that were born on day 1 to cohort 2)

In [ ]:
with open("input_2021/6.txt") as f:
    day = 1
    fishList = [int(fish) for fish in f.read().strip().split(',')]
    fishDict = {i:fishList.count(i) for i in range(7)}
    for day in range(256):
        reproducing_cohort = day % 7
        try:
            fishDict[(reproducing_cohort+2) % 7] += fishDict[reproducing_cohort+8]
        except Exception as e:
            pass
        fishDict[reproducing_cohort+8] = fishDict[reproducing_cohort]
        

print(fishDict)
sum([v for k,v in fishDict.items()])

## Day 5

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 5)

### puzzle 1

In [ ]:
import numpy as np
field = np.zeros([1000, 1000])
with open("input_2021/5.txt") as f:
    for line in f:
        x1, y1, x2, y2 = [int(coord) for coord in line.strip().replace(" -> ", ",").split(",")]
        is_horizontal = y1 == y2
        is_vertical = x1 == x2  
        if is_horizontal:
            xs = range(min(x1, x2), max(x1, x2)+1)
            for x in xs:
                field[y1, x] += 1
        if is_vertical:
            ys = range(min(y1, y2), max(y1, y2)+1)
            for y in ys:
                field[y, x1] += 1
overlaps = field > 1
print(overlaps.sum())
         

        

### puzzle 2

In [ ]:
import numpy as np
field = np.zeros([1000, 1000])
with open("input_2021/5.txt") as f:
    for line in f:
        x1, y1, x2, y2 = [int(coord) for coord in line.strip().replace(" -> ", ",").split(",")]
        if x1 > x2:
            x_direction = -1
            x2 -= 1
        else:
            x_direction = 1
            x2 += 1
        if y1 > y2:
            y_direction = -1
            y2 -= 1
        else:
            y_direction = 1
            y2 += 1
        x_coords = [x for x in range(x1, x2, x_direction)]
        y_coords = [y for y in range(y1, y2, y_direction)]
        if len(x_coords) == 1:
            x_coords *= len(y_coords)
        if len(y_coords) == 1:
            y_coords *= len(x_coords)
        covered_coords = [coords for coords in zip(x_coords, y_coords)]
        for covered_x, covered_y in covered_coords:
                field[covered_y, covered_x] += 1
overlaps = field > 1
print(overlaps.sum())
        


## Day 4

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 4)

### puzzle 1

In [ ]:
import numpy as np

def check_winner(board):
    column_win = np.isnan(board).all(axis=0).any()
    row_win = np.isnan(board).all(axis=1).any()
    return column_win or row_win


with open("input_2021/4.txt") as f:
    bingo_numbers = [int(num) for num in next(f).strip().split(",")]
    boardStrings = f.read().split("\n\n")
    boardLists = [boardString.strip().split("\n") for boardString in boardStrings]
    boardArrays = []
    for boardList in boardLists:
        boardNestedList = []
        for linestring in boardList:
            lineList = [int(num) for num in linestring.split()]
            boardNestedList.append(lineList)
        boardArray = np.array(boardNestedList, dtype='float')
        boardArrays.append(boardArray)

for i, number in enumerate(bingo_numbers):
    for array in boardArrays:
        if number in array:
            array[array==number] = np.nan
            if check_winner(array):
                print(array)
                break
    if check_winner(array):
        break
np.nansum(array) * number


### puzzle 2

In [ ]:
with open("input_2021/4.txt") as f:
    bingo_numbers = [int(num) for num in next(f).strip().split(",")]
    boardStrings = f.read().split("\n\n")
    boardLists = [boardString.strip().split("\n") for boardString in boardStrings]
    boardArrays = []
    for boardList in boardLists:
        boardNestedList = []
        for linestring in boardList:
            lineList = [int(num) for num in linestring.split()]
            boardNestedList.append(lineList)
        boardArray = np.array(boardNestedList, dtype='float')
        boardArrays.append(boardArray)

boardDict = {i: array for i, array in enumerate(boardArrays)}

# Everytime you find a winner, remove it from the board dictionary.
# The first time the board dictionary is empty, that's the last winning board.
for number in bingo_numbers:
    for i, array in enumerate(boardArrays):
        if number in array:
            array[array==number] = np.nan
            if check_winner(array):
                boardDict.pop(i, None)
                if len(boardDict) == 0:
                    print(np.nansum(array) * number)
                    break
    if len(boardDict) == 0:
        break



### refactor puzzle 2

breaking out of the inner loop isn't natively supported in Python. The original strategy of checking the break condition 2x works, but isn't efficient. Better is to put the loops in a function, and use return instead of break.

In [ ]:
with open("input_2021/4.txt") as f:
    bingo_numbers = [int(num) for num in next(f).strip().split(",")]
    boardStrings = f.read().split("\n\n")
    boardLists = [boardString.strip().split("\n") for boardString in boardStrings]
    boardArrays = []
    for boardList in boardLists:
        boardNestedList = []
        for linestring in boardList:
            lineList = [int(num) for num in linestring.split()]
            boardNestedList.append(lineList)
        boardArray = np.array(boardNestedList, dtype='float')
        boardArrays.append(boardArray)

boardDict = {i: array for i, array in enumerate(boardArrays)}

# Everytime you find a winner, remove it from the board dictionary.
# The first time the board dictionary is empty, that's the last winning board.
def win_check():
    for number in bingo_numbers:
        for i, array in enumerate(boardArrays):
            if number in array:
                array[array==number] = np.nan
                if check_winner(array):
                    boardDict.pop(i, None)
                    if len(boardDict) == 0:
                        return np.nansum(array) * number
win_check()

## Day 3

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 3)

### puzzle 1

In [ ]:
from collections import Counter
with open("input_2021/3.txt") as f:
    places_dict = {}
    for line in f:
        for i, char in enumerate(line.rstrip()):
            if i in places_dict.keys():
                places_dict[i].append(char)
            else:
                places_dict[i] = [char]
gamma_bin = ""
epsilon_bin = ""
for place, val in places_dict.items():
     most_common = Counter(val).most_common()
     gamma_bin += most_common[0][0]
     epsilon_bin += most_common[-1][0]



int(gamma_bin, 2) * int(epsilon_bin, 2)



### puzzle 2

In [ ]:
from collections import Counter
def o_searcher(vals, place=0):
    separated = {'0': [], '1':[]}
    for line in vals:
        separated[line[place]].append(line)
    if len(separated['0']) > len(separated['1']):
        if len(separated['0']) == 1:
            oxygen = separated['0'][0]
            return oxygen
        oxygen = o_searcher(separated['0'], place+1)
    else:
        if len(separated['1']) == 1:
            oxygen = separated['1'][0]
            return oxygen
        oxygen = o_searcher(separated['1'], place+1)
    return oxygen

def c_searcher(vals, place=0):
    separated = {'0': [], '1':[]}
    for line in vals:
        separated[line[place]].append(line)
    if len(separated['0']) <= len(separated['1']):
        if len(separated['0']) == 1:
            co2 = separated['0'][0]
            return co2
        co2 = c_searcher(separated['0'], place+1)
    else:
        if len(separated['1']) == 1:
            co2 = separated['1'][0]
            return co2
        co2 = c_searcher(separated['1'], place+1)
    return co2

with open("input_2021/3.txt") as f:
    vals = f.read().strip().split("\n")
    oxygen = o_searcher(vals)
    co2 = c_searcher(vals)
int(oxygen, 2) * int(co2, 2)

## Day 2

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 2)

### puzzle 1

In [ ]:
with open("input_2021/2.txt") as f:
    hpos = 0
    depth = 0
    for line in f:
        command = line.strip().split()
        if command[0] == "forward":
            hpos += int(command[1])
        elif command[0] == "up":
            depth -= int(command[1])
        elif command[0] == "down":
            depth += int(command[1])
        else:
            print('bad')
hpos * depth

### puzzle 2

In [ ]:
with open("input_2021/2.txt") as f:
    hpos = 0
    depth = 0
    aim = 0
    for line in f:
        command, raw_val = line.strip().split()
        val = int(raw_val)
        if command == "forward":
            hpos += val
            depth += val * aim
        elif command == "up":
            aim -= val
        else: aim += val
hpos * depth

## Day 1

Task:

* Compare values in a text file to values above
* First line by line, then in overlapping moving windows

Reflection:

For the moving window, my initial strategy was to assign values fo every measure in the window (m1, m2, m3) then move them up (so the new m2 is the old m3, the new m1 is the old m2). This works, but it seemed like there should be a more elegant solution.

In [ ]:
import aoc
aoc.read_input("cookie.txt", "input_2021", 2021, 1)

### puzzle 1

In [ ]:
with open("input_2021/1.txt") as f:
    increases = -1
    previous_depth = 0
    for line in f:
        depth = int(line.rstrip())
        if depth > previous_depth:
            increases += 1
        previous_depth = depth
increases


### puzzle 2

In [ ]:
with open("input_2021/1.txt") as f:
    m1 = int(next(f).rstrip())
    m2 = int(next(f).rstrip())
    g1_sum = 0
    increases = -1
    for line in f:
        m3 = int(line.rstrip())
        g2_sum = m1+m2+m3
        if g2_sum > g1_sum:
            increases += 1
        m1 = m2
        m2 = m3
        g1_sum = g2_sum

increases

### puzzle 1 refactor

Instead of artificially lowering the value of the `increases` variable to account for the fact that the first value should not be counted as an increase, read the first value using `next` outside the main loop.

In [ ]:
with open("input_2021/1.txt") as f:
    increases = 0
    previous_depth = int(next(f).rstrip())
    for line in f:
        depth = int(line.rstrip())
        if depth > previous_depth:
            increases += 1
        previous_depth = depth
increases

### puzzle 2 refactor

In [ ]:
with open("input_2021/1.txt") as f:
    m1 = int(next(f).rstrip())
    m2 = int(next(f).rstrip())
    m3 = int(next(f).rstrip())
    g1_sum = m1+m2+m3
    increases = 0
    for line in f:
        m1 = m2
        m2 = m3
        m3 = int(line.rstrip())
        g2_sum = m1+m2+m3
        if g2_sum > g1_sum:
            increases += 1
        g1_sum = g2_sum
increases